In [1]:
import tensorflow as tf

In [2]:
tf.reset_default_graph()

Vamos carregar o modelo que já temos treinado.

In [3]:
restore_saver = tf.train.import_meta_graph("./mnist-0-4-tf.model.meta")

In [4]:
X = tf.get_default_graph().get_tensor_by_name("X:0")
y = tf.get_default_graph().get_tensor_by_name("y:0")
loss = tf.get_default_graph().get_tensor_by_name("loss:0")
y_proba = tf.get_default_graph().get_tensor_by_name("y_proba:0")
logits = y_proba.op.inputs[0]
accuracy = tf.get_default_graph().get_tensor_by_name("accuracy:0")

Para que possamos congelar as camadas inferiores, vamos remocer suas variáveis da lista de variáveis treinávis (trainables) do otimizador, deixando apenas a camada saída ocmo treinável.

In [5]:
learning_rate = 0.001

output_layer_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="logits")
optimizer = tf.train.AdamOptimizer(learning_rate, name="Adam2")
training_op = optimizer.minimize(loss, var_list=output_layer_vars)

Instructions for updating:
Colocations handled automatically by placer.


In [6]:
correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

In [7]:
init = tf.global_variables_initializer()
five_frozen_saver = tf.train.Saver()

Agora vamos treinar essa nova DNN em dígitos de 5 a 9 (usamos a mesma camada de saída pois ainda temos o mesmo formato, 5 dígitos)

Vamos carregar os dados, note que removemos 5 das labels porque o tensorflow espera inteiros de 0 a n_classes - 1

In [8]:
import numpy as np

In [9]:
mnist = tf.contrib.learn.datasets.load_dataset("mnist")
train_data = mnist.train.images # Returns np.array
train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
eval_data = mnist.test.images # Returns np.array
eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

X_valid, X_train = train_data[:5000], train_data[5000:]
y_valid, y_train = train_labels[:5000], train_labels[5000:]

X_train1 = X_train[y_train < 5]
y_train1 = y_train[y_train < 5]
X_valid1 = X_valid[y_valid < 5]
y_valid1 = y_valid[y_valid < 5]
X_test1 = eval_data[eval_labels < 5]
y_test1 = eval_labels[eval_labels < 5]

X_train2 = X_train[y_train >= 5]
y_train2 = y_train[y_train >= 5] - 5
X_valid2 = X_valid[y_valid >= 5]
y_valid2 = y_valid[y_valid >= 5] - 5
X_test2 = eval_data[eval_labels >= 5]
y_test2 = eval_labels[eval_labels >= 5] - 5


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use tf.data.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST-data\train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST-data\train-

In [10]:
n_epochs = 1000
batch_size = 20

max_checks_without_progress = 20
checks_without_progress = 0
best_loss = np.infty

In [13]:
with tf.Session() as sess:
    init.run()
    restore_saver.restore(sess, "./mnist-0-4-tf.model")
    for var in output_layer_vars:
        var.initializer.run()
    
    for epoch in range(n_epochs):
        rnd_idx = np.random.permutation(len(X_train2))
        for rnd_indices in np.array_split(rnd_idx, len(X_train2)//batch_size):
            X_batch, y_batch = X_train2[rnd_indices], y_train2[rnd_indices]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        loss_val, acc_val = sess.run([loss, accuracy], feed_dict={X: X_valid2, y:y_valid2})
        if loss_val < best_loss:
            save_path = five_frozen_saver.save(sess, "./my_mnist_model_5_to_9_five_frozen")
            best_loss = loss_val
            checks_without_progress = 0
        else:
            checks_without_progress += 1
            if checks_without_progress > max_checks_without_progress:
                print("Early stopping!")
                break
        print("{}\tValidation loss: {:.6f}\tBest loss: {:.6f}\tAccuracy: {:.2f}%".format(
            epoch, loss_val, best_loss, acc_val * 100))

with tf.Session() as sess:
    five_frozen_saver.restore(sess, "./my_mnist_model_5_to_9_five_frozen")
    acc_test = accuracy.eval(feed_dict={X: X_test2, y: y_test2})
    print("Final test accuracy: {:.2f}%".format(acc_test * 100))

INFO:tensorflow:Restoring parameters from ./mnist-0-4-tf.model
0	Validation loss: 0.422767	Best loss: 0.422767	Accuracy: 85.99%
1	Validation loss: 0.359880	Best loss: 0.359880	Accuracy: 89.08%
2	Validation loss: 0.329546	Best loss: 0.329546	Accuracy: 89.78%
3	Validation loss: 0.314028	Best loss: 0.314028	Accuracy: 90.28%
4	Validation loss: 0.314009	Best loss: 0.314009	Accuracy: 89.82%
5	Validation loss: 0.301106	Best loss: 0.301106	Accuracy: 90.11%
6	Validation loss: 0.294571	Best loss: 0.294571	Accuracy: 90.65%
7	Validation loss: 0.286487	Best loss: 0.286487	Accuracy: 90.73%
8	Validation loss: 0.299072	Best loss: 0.286487	Accuracy: 90.23%
9	Validation loss: 0.287135	Best loss: 0.286487	Accuracy: 91.02%
10	Validation loss: 0.296370	Best loss: 0.286487	Accuracy: 90.03%
11	Validation loss: 0.284234	Best loss: 0.284234	Accuracy: 90.94%
12	Validation loss: 0.284318	Best loss: 0.284234	Accuracy: 91.02%
13	Validation loss: 0.288726	Best loss: 0.284234	Accuracy: 90.52%
14	Validation loss: 0.2

Neste exemplo, a única camada treinável foi a a de saída, ou seja, utilizamos o conhecimento do modelo treinado em dígitos de 0 a 4 e ajustamos apenas uma camada para que ele pudesse aprender a reconhecer outros dígitos.

Vamos adicionar uma camada no topo da última camada oculta da rede e retreinar o modelo com uma nova softmax.

In [66]:
from tensorflow.contrib.layers import fully_connected

In [67]:
he_init = tf.contrib.layers.variance_scaling_initializer()

In [68]:
tf.reset_default_graph()
n_outputs = 5
restore_saver = tf.train.import_meta_graph("./mnist-0-4-tf.model.meta")

In [69]:
X = tf.get_default_graph().get_tensor_by_name("X:0")
y = tf.get_default_graph().get_tensor_by_name("y:0")

In [70]:
hidden5_out = tf.get_default_graph().get_tensor_by_name("hidden5/Elu:0")

In [71]:
hidden_6 = fully_connected(hidden5_out, 100, activation_fn=tf.nn.elu, 
                            weights_initializer=he_init, scope="hidden6")

In [72]:
logits = fully_connected(hidden_6, 5, activation_fn=None, weights_initializer=he_init, 
                         scope="new_logits")

In [73]:
Y_proba = tf.nn.softmax(logits)
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
loss = tf.reduce_mean(xentropy)
correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

In [74]:
learning_rate = 0.001

output_layer_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="hidden6|new_logits")
optimizer = tf.train.AdamOptimizer(learning_rate, name="Adam2")
training_op = optimizer.minimize(loss, var_list=output_layer_vars)

init = tf.global_variables_initializer()
four_frozen_saver = tf.train.Saver()

In [82]:
n_epochs = 1000
batch_size = 20

max_checks_without_progress = 20
checks_without_progress = 0
best_loss = np.infty

In [83]:
with tf.Session() as sess:
    init.run()
    restore_saver.restore(sess, "./mnist-0-4-tf.model")
    for var in output_layer_vars:
        var.initializer.run()
    
    for epoch in range(n_epochs):
        rnd_idx = np.random.permutation(len(X_train2))
        for rnd_indices in np.array_split(rnd_idx, len(X_train2)//batch_size):
            X_batch, y_batch = X_train2[rnd_indices], y_train2[rnd_indices]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        loss_val, acc_val = sess.run([loss, accuracy], feed_dict={X: X_valid2, y:y_valid2})
        if loss_val < best_loss:
            save_path = four_frozen_saver.save(sess, "./my_mnist_model_four_frozen")
            best_loss = loss_val
            checks_without_progress = 0
        else:
            checks_without_progress += 1
            if checks_without_progress > max_checks_without_progress:
                print("Early stopping!")
                break
        print("{}\tValidation loss: {:.6f}\tBest loss: {:.6f}\tAccuracy: {:.2f}%".format(
            epoch, loss_val, best_loss, acc_val * 100))

with tf.Session() as sess:
    four_frozen_saver.restore(sess, "./my_mnist_model_four_frozen")
    acc_test = accuracy.eval(feed_dict={X: X_test2, y: y_test2})
    print("Final test accuracy: {:.2f}%".format(acc_test * 100))

INFO:tensorflow:Restoring parameters from ./mnist-0-4-tf.model
0	Validation loss: 0.295949	Best loss: 0.295949	Accuracy: 90.77%
1	Validation loss: 0.259078	Best loss: 0.259078	Accuracy: 91.43%
2	Validation loss: 0.217295	Best loss: 0.217295	Accuracy: 92.46%
3	Validation loss: 0.224532	Best loss: 0.217295	Accuracy: 92.38%
4	Validation loss: 0.187246	Best loss: 0.187246	Accuracy: 93.41%
5	Validation loss: 0.178598	Best loss: 0.178598	Accuracy: 93.57%
6	Validation loss: 0.197313	Best loss: 0.178598	Accuracy: 93.00%
7	Validation loss: 0.181959	Best loss: 0.178598	Accuracy: 93.65%
8	Validation loss: 0.177878	Best loss: 0.177878	Accuracy: 93.90%
9	Validation loss: 0.163134	Best loss: 0.163134	Accuracy: 94.19%
10	Validation loss: 0.164856	Best loss: 0.163134	Accuracy: 94.56%
11	Validation loss: 0.186183	Best loss: 0.163134	Accuracy: 94.07%
12	Validation loss: 0.177134	Best loss: 0.163134	Accuracy: 93.57%
13	Validation loss: 0.151317	Best loss: 0.151317	Accuracy: 94.81%
14	Validation loss: 0.1